# Deep cortical reinforcement learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [2]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [3]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [4]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [5]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [6]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [7]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [8]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.349380847965382 -2.602122511534467
actions: 1 0
rewards: 1.0 1.0


In [9]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [10]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [12]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [13]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [14]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    return states, actions, targetQs

In [15]:
# How to use batch-norm
#   x_norm = tf.layers.batch_normalization(x, training=training)

#   # ...

#   update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#   with tf.control_dependencies(update_ops):
#     train_op = optimizer.minimize(loss)

In [16]:
# training: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
# Whether to return the output in: 
# training mode (normalized with statistics of the current batch) or 
# inference mode (normalized with moving statistics). 
# NOTE: make sure to set this parameter correctly, or else your training/inference will not work properly.

In [83]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [84]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [85]:
def model_loss(action_size, hidden_size, states, actions, targetQs):
    # G
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    g_loss = tf.reduce_mean(neg_log_prob_actions * targetQs)
    #g_loss = -tf.reduce_mean(Qs)
    
    # D
    Qs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    d_loss = tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, Qs, g_loss, d_loss

In [86]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [87]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, targetQs=self.targetQs) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

In [88]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [89]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1000, 4) actions:(1000,)
action size:2


In [96]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
# state_size = 37
# state_size_ = (84, 84, 3)
state_size = 4
action_size = 2
hidden_size = 64             # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
gamma = 0.99                   # future reward discount
memory_size = 1000            # memory capacity
batch_size = 1000             # experience mini-batch size

In [97]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [98]:
state = env.reset()
#for _ in range(batch_size):
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list, gloss_list, dloss_list = [], [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        gloss_batch, dloss_batch = [], []
        state = env.reset()

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            total_reward += reward
            state = next_state

            # Training
            #batch = memory.sample(batch_size)
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            nextQs_logits = sess.run(model.Qs_logits, feed_dict = {model.states: next_states})
            #nextQs = np.max(nextQs_logits, axis=1) * (1-dones)
            nextQs = nextQs_logits.reshape([-1]) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            gloss, dloss, _, _ = sess.run([model.g_loss, model.d_loss, model.g_opt, model.d_opt],
                                            feed_dict = {model.states: states, 
                                                         model.actions: actions,
                                                         model.targetQs: targetQs})
            gloss_batch.append(gloss)
            dloss_batch.append(dloss)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{}'.format(total_reward),
              'gloss:{:.4f}'.format(np.mean(gloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        gloss_list.append([ep, np.mean(gloss_batch)])
        dloss_list.append([ep, np.mean(dloss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0 gloss:0.6995 dloss:1.0029 exploreP:0.9983
Episode:1 meanR:14.0000 R:11.0 gloss:0.7397 dloss:0.9983 exploreP:0.9972
Episode:2 meanR:13.6667 R:13.0 gloss:0.7764 dloss:0.9974 exploreP:0.9959
Episode:3 meanR:14.7500 R:18.0 gloss:0.8250 dloss:1.0005 exploreP:0.9942
Episode:4 meanR:14.0000 R:11.0 gloss:0.8725 dloss:1.0073 exploreP:0.9931
Episode:5 meanR:13.3333 R:10.0 gloss:0.9091 dloss:1.0145 exploreP:0.9921
Episode:6 meanR:15.4286 R:28.0 gloss:0.9780 dloss:1.0336 exploreP:0.9894
Episode:7 meanR:15.5000 R:16.0 gloss:1.0668 dloss:1.0634 exploreP:0.9878
Episode:8 meanR:17.5556 R:34.0 gloss:1.1835 dloss:1.1067 exploreP:0.9845
Episode:9 meanR:18.0000 R:22.0 gloss:1.3335 dloss:1.1752 exploreP:0.9823
Episode:10 meanR:19.6364 R:36.0 gloss:1.5212 dloss:1.2884 exploreP:0.9788
Episode:11 meanR:18.8333 R:10.0 gloss:1.6844 dloss:1.4028 exploreP:0.9779
Episode:12 meanR:18.6923 R:17.0 gloss:1.7944 dloss:1.5063 exploreP:0.9762
Episode:13 meanR:19.7143 R:33.0 gloss:2.0090 dlo

Episode:109 meanR:22.9400 R:23.0 gloss:12.7152 dloss:15.2435 exploreP:0.7830
Episode:110 meanR:22.9900 R:41.0 gloss:12.6760 dloss:15.1259 exploreP:0.7799
Episode:111 meanR:23.1300 R:24.0 gloss:12.8144 dloss:15.1220 exploreP:0.7780
Episode:112 meanR:23.1600 R:20.0 gloss:12.8668 dloss:15.2661 exploreP:0.7765
Episode:113 meanR:23.1900 R:36.0 gloss:13.0036 dloss:15.2785 exploreP:0.7737
Episode:114 meanR:23.3300 R:26.0 gloss:13.1341 dloss:15.3569 exploreP:0.7717
Episode:115 meanR:23.1700 R:19.0 gloss:13.2686 dloss:15.8176 exploreP:0.7703
Episode:116 meanR:23.1300 R:15.0 gloss:13.2091 dloss:16.1829 exploreP:0.7692
Episode:117 meanR:23.1600 R:11.0 gloss:13.0833 dloss:15.8950 exploreP:0.7683
Episode:118 meanR:23.0900 R:25.0 gloss:12.9167 dloss:15.8223 exploreP:0.7664
Episode:119 meanR:23.0300 R:10.0 gloss:12.7687 dloss:15.7991 exploreP:0.7657
Episode:120 meanR:23.0400 R:32.0 gloss:12.4883 dloss:15.2803 exploreP:0.7633
Episode:121 meanR:23.2900 R:37.0 gloss:12.4689 dloss:14.8850 exploreP:0.7605

Episode:216 meanR:24.5400 R:34.0 gloss:13.1648 dloss:16.6589 exploreP:0.6040
Episode:217 meanR:24.6000 R:17.0 gloss:13.2167 dloss:16.7117 exploreP:0.6029
Episode:218 meanR:24.4600 R:11.0 gloss:13.1961 dloss:16.9715 exploreP:0.6023
Episode:219 meanR:24.4700 R:11.0 gloss:13.0788 dloss:16.7509 exploreP:0.6016
Episode:220 meanR:24.2700 R:12.0 gloss:12.8684 dloss:16.7371 exploreP:0.6009
Episode:221 meanR:24.2700 R:37.0 gloss:12.6662 dloss:16.1550 exploreP:0.5988
Episode:222 meanR:24.3100 R:14.0 gloss:12.8151 dloss:16.5046 exploreP:0.5979
Episode:223 meanR:24.3200 R:32.0 gloss:12.7402 dloss:16.3624 exploreP:0.5960
Episode:224 meanR:24.5000 R:35.0 gloss:12.9939 dloss:16.4979 exploreP:0.5940
Episode:225 meanR:24.5000 R:10.0 gloss:13.3630 dloss:17.2178 exploreP:0.5934
Episode:226 meanR:25.1300 R:74.0 gloss:13.3023 dloss:16.8726 exploreP:0.5891
Episode:227 meanR:25.2400 R:29.0 gloss:13.6339 dloss:17.4952 exploreP:0.5874
Episode:228 meanR:25.2000 R:21.0 gloss:13.4390 dloss:16.9777 exploreP:0.5862

Episode:323 meanR:36.6800 R:72.0 gloss:13.6003 dloss:20.3463 exploreP:0.4161
Episode:324 meanR:37.0300 R:70.0 gloss:13.5597 dloss:20.6116 exploreP:0.4133
Episode:325 meanR:37.2800 R:35.0 gloss:13.2605 dloss:20.5697 exploreP:0.4119
Episode:326 meanR:37.4100 R:87.0 gloss:13.5185 dloss:20.5977 exploreP:0.4084
Episode:327 meanR:37.8600 R:74.0 gloss:13.5631 dloss:20.9808 exploreP:0.4054
Episode:328 meanR:38.2900 R:64.0 gloss:13.8831 dloss:20.9112 exploreP:0.4029
Episode:329 meanR:39.0000 R:83.0 gloss:14.5756 dloss:21.9077 exploreP:0.3997
Episode:330 meanR:38.8200 R:21.0 gloss:14.5045 dloss:22.3210 exploreP:0.3989
Episode:331 meanR:39.0200 R:56.0 gloss:14.2327 dloss:21.4943 exploreP:0.3967
Episode:332 meanR:38.8300 R:24.0 gloss:14.7648 dloss:21.8332 exploreP:0.3958
Episode:333 meanR:39.1800 R:50.0 gloss:14.4786 dloss:21.5456 exploreP:0.3938
Episode:334 meanR:39.7200 R:66.0 gloss:13.9311 dloss:21.4666 exploreP:0.3913
Episode:335 meanR:39.6600 R:22.0 gloss:13.7931 dloss:21.2300 exploreP:0.3905

Episode:430 meanR:35.5500 R:43.0 gloss:9.2953 dloss:17.5232 exploreP:0.2825
Episode:431 meanR:35.4900 R:50.0 gloss:9.2967 dloss:17.6757 exploreP:0.2812
Episode:432 meanR:35.8700 R:62.0 gloss:9.4646 dloss:18.0042 exploreP:0.2795
Episode:433 meanR:35.6200 R:25.0 gloss:9.5457 dloss:18.3695 exploreP:0.2788
Episode:434 meanR:35.3100 R:35.0 gloss:9.5134 dloss:18.0277 exploreP:0.2779
Episode:435 meanR:35.4400 R:35.0 gloss:9.5653 dloss:18.1101 exploreP:0.2769
Episode:436 meanR:35.5600 R:46.0 gloss:9.6875 dloss:18.5578 exploreP:0.2757
Episode:437 meanR:35.5500 R:15.0 gloss:9.5815 dloss:18.6030 exploreP:0.2753
Episode:438 meanR:35.9300 R:50.0 gloss:9.2979 dloss:18.0256 exploreP:0.2740
Episode:439 meanR:35.9500 R:33.0 gloss:9.3050 dloss:18.2557 exploreP:0.2731
Episode:440 meanR:35.5800 R:13.0 gloss:9.4716 dloss:18.6649 exploreP:0.2728
Episode:441 meanR:35.3500 R:42.0 gloss:9.4406 dloss:17.9791 exploreP:0.2717
Episode:442 meanR:35.3800 R:39.0 gloss:9.2838 dloss:17.8713 exploreP:0.2707
Episode:443 

...

Episode:3520 meanR:41.8200 R:89.0 gloss:0.9338 dloss:18.9998 exploreP:0.0100
Episode:3521 meanR:41.7800 R:25.0 gloss:0.9092 dloss:19.5113 exploreP:0.0100
Episode:3522 meanR:41.5900 R:24.0 gloss:0.9060 dloss:19.2301 exploreP:0.0100
Episode:3523 meanR:41.7500 R:31.0 gloss:0.8956 dloss:18.9679 exploreP:0.0100
Episode:3524 meanR:41.7500 R:255.0 gloss:0.8464 dloss:19.6779 exploreP:0.0100
Episode:3525 meanR:41.1500 R:22.0 gloss:0.8238 dloss:20.7384 exploreP:0.0100
Episode:3526 meanR:41.2500 R:37.0 gloss:0.7818 dloss:19.8512 exploreP:0.0100
Episode:3527 meanR:41.1600 R:10.0 gloss:0.7912 dloss:20.5839 exploreP:0.0100
Episode:3528 meanR:40.9200 R:14.0 gloss:0.7523 dloss:20.2593 exploreP:0.0100
Episode:3529 meanR:41.1600 R:53.0 gloss:0.6595 dloss:19.3297 exploreP:0.0100
Episode:3530 meanR:40.9300 R:19.0 gloss:0.6977 dloss:20.6231 exploreP:0.0100
Episode:3531 meanR:41.1300 R:55.0 gloss:0.6386 dloss:19.9474 exploreP:0.0100
Episode:3532 meanR:41.1500 R:43.0 gloss:0.5961 dloss:20.2770 exploreP:0.010

Episode:3627 meanR:39.4000 R:56.0 gloss:1.0333 dloss:22.7343 exploreP:0.0100
Episode:3628 meanR:39.5600 R:30.0 gloss:0.9883 dloss:23.5463 exploreP:0.0100
Episode:3629 meanR:39.1500 R:12.0 gloss:0.9445 dloss:25.0330 exploreP:0.0100
Episode:3630 meanR:39.4000 R:44.0 gloss:0.8256 dloss:23.6575 exploreP:0.0100
Episode:3631 meanR:39.0200 R:17.0 gloss:0.7616 dloss:23.9016 exploreP:0.0100
Episode:3632 meanR:39.2000 R:61.0 gloss:0.8572 dloss:22.6978 exploreP:0.0100
Episode:3633 meanR:39.2700 R:32.0 gloss:1.1073 dloss:22.6654 exploreP:0.0100
Episode:3634 meanR:39.8600 R:70.0 gloss:0.9917 dloss:21.8671 exploreP:0.0100
Episode:3635 meanR:40.0800 R:68.0 gloss:0.9871 dloss:21.3630 exploreP:0.0100
Episode:3636 meanR:39.9100 R:18.0 gloss:1.0263 dloss:21.2522 exploreP:0.0100
Episode:3637 meanR:40.1100 R:41.0 gloss:1.0453 dloss:22.3116 exploreP:0.0100
Episode:3638 meanR:39.8400 R:33.0 gloss:1.0205 dloss:21.8841 exploreP:0.0100
Episode:3639 meanR:39.9700 R:26.0 gloss:1.0101 dloss:22.1621 exploreP:0.0100

Episode:3734 meanR:38.3000 R:35.0 gloss:0.7122 dloss:18.7060 exploreP:0.0100
Episode:3735 meanR:38.2500 R:63.0 gloss:0.8715 dloss:18.8764 exploreP:0.0100
Episode:3736 meanR:38.5400 R:47.0 gloss:0.9531 dloss:19.2040 exploreP:0.0100
Episode:3737 meanR:38.3400 R:21.0 gloss:0.8623 dloss:19.4312 exploreP:0.0100
Episode:3738 meanR:38.3600 R:35.0 gloss:0.8028 dloss:19.2941 exploreP:0.0100
Episode:3739 meanR:38.4400 R:34.0 gloss:1.0790 dloss:18.8854 exploreP:0.0100
Episode:3740 meanR:37.9800 R:9.0 gloss:1.0059 dloss:19.1681 exploreP:0.0100
Episode:3741 meanR:38.2200 R:37.0 gloss:0.8713 dloss:18.7506 exploreP:0.0100
Episode:3742 meanR:38.3100 R:24.0 gloss:0.7689 dloss:18.3323 exploreP:0.0100
Episode:3743 meanR:38.2200 R:27.0 gloss:0.7315 dloss:18.2731 exploreP:0.0100
Episode:3744 meanR:36.7200 R:49.0 gloss:0.6957 dloss:18.0893 exploreP:0.0100
Episode:3745 meanR:36.6800 R:9.0 gloss:0.6777 dloss:18.9781 exploreP:0.0100
Episode:3746 meanR:36.8500 R:46.0 gloss:0.6546 dloss:17.8927 exploreP:0.0100
E

Episode:3841 meanR:41.8200 R:60.0 gloss:0.3099 dloss:18.8478 exploreP:0.0100
Episode:3842 meanR:41.8400 R:26.0 gloss:0.2998 dloss:19.2896 exploreP:0.0100
Episode:3843 meanR:42.1100 R:54.0 gloss:0.2790 dloss:19.2327 exploreP:0.0100
Episode:3844 meanR:41.8300 R:21.0 gloss:0.2681 dloss:19.7056 exploreP:0.0100
Episode:3845 meanR:42.0900 R:35.0 gloss:0.2608 dloss:19.0295 exploreP:0.0100
Episode:3846 meanR:44.0900 R:246.0 gloss:0.8679 dloss:19.2874 exploreP:0.0100
Episode:3847 meanR:46.1400 R:232.0 gloss:1.5230 dloss:21.2051 exploreP:0.0100
Episode:3848 meanR:46.8900 R:120.0 gloss:1.6547 dloss:23.0350 exploreP:0.0100
Episode:3849 meanR:48.4400 R:167.0 gloss:1.9876 dloss:23.1954 exploreP:0.0100
Episode:3850 meanR:48.5500 R:39.0 gloss:2.4172 dloss:23.3499 exploreP:0.0100
Episode:3851 meanR:49.5600 R:118.0 gloss:2.5545 dloss:24.0172 exploreP:0.0100
Episode:3852 meanR:49.5100 R:24.0 gloss:2.6720 dloss:25.6164 exploreP:0.0100
Episode:3853 meanR:49.1300 R:10.0 gloss:2.5142 dloss:24.7594 exploreP:0

Episode:3948 meanR:39.1000 R:37.0 gloss:0.5565 dloss:16.4845 exploreP:0.0100
Episode:3949 meanR:37.8500 R:42.0 gloss:0.5245 dloss:16.7502 exploreP:0.0100
Episode:3950 meanR:37.7500 R:29.0 gloss:0.6361 dloss:16.5817 exploreP:0.0100
Episode:3951 meanR:36.9900 R:42.0 gloss:0.6023 dloss:16.2613 exploreP:0.0100
Episode:3952 meanR:37.0500 R:30.0 gloss:0.5194 dloss:16.5566 exploreP:0.0100
Episode:3953 meanR:37.3400 R:39.0 gloss:0.4186 dloss:16.1355 exploreP:0.0100
Episode:3954 meanR:36.9400 R:10.0 gloss:0.3931 dloss:16.3925 exploreP:0.0100
Episode:3955 meanR:37.0600 R:72.0 gloss:0.3678 dloss:16.2291 exploreP:0.0100
Episode:3956 meanR:37.1500 R:18.0 gloss:0.3280 dloss:16.4084 exploreP:0.0100
Episode:3957 meanR:36.2000 R:22.0 gloss:0.3169 dloss:16.3457 exploreP:0.0100
Episode:3958 meanR:36.9100 R:94.0 gloss:0.2716 dloss:16.6196 exploreP:0.0100
Episode:3959 meanR:34.8600 R:13.0 gloss:0.2455 dloss:17.1326 exploreP:0.0100
Episode:3960 meanR:34.8400 R:20.0 gloss:0.2423 dloss:16.6034 exploreP:0.0100

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [36]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.